# Mirsha VetEpiGeoAI: End-to-End User Workflow

This notebook serves as an interactive user manual and demonstration of the complete workflow for the Mirsha VetEpiGeoAI platform. It simulates the entire lifecycle of an animal health event, from initial setup to strategic analysis.

**Objective:** To provide a clear, step-by-step, and reproducible guide for users, trainers, and developers.

## Phase 1: Initial Platform Setup

This phase is performed once to initialize the environment and populate the database with both structural (GIS) and example (case) data. This section simulates the actions a platform administrator would take.

In [ ]:
# Step 1: Clone the Repository and Install Dependencies
import os
import sys

REPO_URL = "https://github.com/bayillag/mirsha-vet-epigeoai.git"
REPO_NAME = "mirsha-vet-epigeoai"

if not os.path.exists(REPO_NAME):
    print(f"Cloning {REPO_URL}...")
    !git clone -q {REPO_URL}
else:
    print("Repository already exists.")

# Change directory into the project and install requirements
os.chdir(REPO_NAME)
print("\n⏳ Installing Python dependencies...")
!pip install -q -r requirements.txt
print("✅ Dependencies installed.")

# Add src to path to allow imports
sys.path.append(os.path.abspath('src'))

In [ ]:
# Step 2: Generate all necessary data files
print("--- Generating Data Files ---")
!python src/scripts/generate_gis_data.py
!python src/scripts/generate_mock_cases.py
!python src/scripts/generate_risk_data.py

In [ ]:
# Step 3: Initialize and Populate the Database
# NOTE: This step assumes the Docker/Codespaces database is running.
# In a local environment, you would need PostgreSQL running.
print("--- Setting up the Database ---")
!python src/scripts/setup_database.py

### Platform is now set up and ready.

To launch the interactive dashboard, you would run `streamlit run app.py` from the terminal. The following sections will simulate the workflow by calling the platform's core functions directly.

## Phase 2: Simulating an Outbreak Response

This section simulates the workflow of a field officer and a regional manager responding to a new event.

In [ ]:
from src.core import database as db
import datetime
import pandas as pd
import streamlit as st # Import for mocking

print("--- Simulating Outbreak Response Workflow ---")

# Step 5: File an Initial Report (Module 1)
print("\nStep 5: Filing new outbreak report...")
success, message = db.add_new_outbreak_report(
    reporter_name='Farmer Kedir',
    woreda_name='Hamer',
    lat=5.10,
    lon=36.60,
    species=['Cattle', 'Goats'],
    complaint='Several animals are weak, drooling, and have blisters on their feet.',
    report_date=datetime.date.today()
)
print(f">> {message}")

# Step 6: Triage and Assign the Report (Module 1)
print("\nStep 6: Triage - Assigning report to an investigator...")
unassigned = db.get_unassigned_reports()
if not unassigned.empty:
    outbreak_to_assign = unassigned.iloc[0]
    outbreak_id = outbreak_to_assign['outbreak_id']
    success, message = db.assign_investigator_to_outbreak(outbreak_id, 'Dr. Alemayehu')
    print(f">> {message}")
else:
    print(">> No unassigned reports found to assign.")
    outbreak_id = None

# Step 7 & 8: Submit Full Investigation & Lab Results (Module 2 & 7)
if outbreak_id:
    print(f"\nStep 7: Submitting full investigation for Outbreak ID {outbreak_id}...")
    form_data = {'first_signs_date': '2023-10-28', 'clinical_signs': ['Drooling saliva', 'Blisters on mouth/feet/udder']}
    line_list = pd.DataFrame([{'Species': 'Cattle', 'Susceptible': 200, 'Cases': 30, 'Deaths': 5, 'Date': datetime.date(2023, 10, 28)}])
    samples = pd.DataFrame([{'Field ID': 'HMR-001A', 'Sample Type': 'Blood (Serum)', 'Laboratory': 'NAHDIC', 'Submission Date': datetime.date.today()}])
    
    success, message = db.submit_full_investigation(outbreak_id, datetime.date.today(), form_data, line_list, samples)
    print(f">> {message}")

    print("\nStep 8: Submitting lab results...")
    sample_details = db.get_sample_by_field_id('HMR-001A')
    if sample_details:
        success, message = db.submit_lab_result(sample_details['id'], outbreak_id, 'Positive', 'FMD-Type O', datetime.date.today())
        print(f">> {message}")
        print("Outbreak status is now automatically 'Confirmed'.")

# Step 9: Add Contact Tracing Links (Module 3)
if outbreak_id:
    print(f"\nStep 9: Adding contact tracing links for Outbreak ID {outbreak_id}...")
    db.add_tracing_link(outbreak_id, 'Turmi Market', 'Market', 4.97, 36.49, datetime.date(2023, 10, 20), 'Animal Movement', 'Trace-back', 'Source of infected animals.')
    db.add_tracing_link(outbreak_id, 'Neighboring Farm', 'Farm', 5.12, 36.62, datetime.date(2023, 10, 29), 'Shared Grazing', 'Trace-forward', 'Potential spread.')
    print(">> 2 contact links added.")

## Phase 3: Simulating Strategic Analysis & Planning

This section simulates the workflow of a national-level epidemiologist using the platform's advanced analytical modules.

In [ ]:
from src.core import analysis as an
from src.core import economics as econ
import matplotlib.pyplot as plt

print("--- Simulating Strategic Analysis Workflow ---")

# Step 10: Analyze the Situation (Module 4)
print("\nStep 10: Running Hotspot Analysis (LISA)...")
woredas_gdf, outbreaks_gdf = db.get_dashboard_data()
fmd_gdf = woredas_gdf[woredas_gdf['disease_name'] == 'FMD']
lisa_results = an.calculate_lisa(fmd_gdf, 'total_cases')
significant_clusters = lisa_results[lisa_results['lisa_p'] <= 0.05]
print("Statistically Significant Clusters for FMD:")
display(significant_clusters[['woreda_name', 'total_cases', 'cluster_type']])

# Step 11: Calculate Economic Impact (Module 11)
if outbreak_id:
    print(f"\nStep 11: Calculating economic cost for Outbreak ID {outbreak_id}...")
    case_data = db.get_cases_for_outbreak(outbreak_id)
    costs = econ.calculate_direct_outbreak_cost(case_data)
    print(f">> Total Estimated Direct Cost: {costs['total_cost']:,.0f} ETB")

# Step 16: Simulate Training Scenario (Module 16)
from src.core import simulation as sim
print("\nStep 16: Simulating a training scenario...")

# Mock Streamlit's session_state for this to run outside the app
class MockSessionState:
    def __init__(self):
        self._state = {}
    def __getattr__(self, name):
        return self._state.get(name)
    def __setattr__(self, name, value):
        self._state[name] = value
    def get(self, name, default=None):
        return self._state.get(name, default)

if 'st' not in locals() or not hasattr(st, 'session_state'):
    # This ensures we can run the notebook standalone
    sim.st = type('MockStreamlit', (), {'session_state': MockSessionState()})()

scenario_df = sim.load_scenario_data('fmd_scenario_2021')
sim.initialize_simulation_state(scenario_df)
sim.get_events_for_day(2) # Advance to day 2
sim.log_trainee_action('Initiate Investigation', 'Bena Tsemay')
sim.score_action('Initiate Investigation', 'Bena Tsemay', sim.st.session_state['events_revealed'])
print(f">> Training action logged. Current score: {sim.st.session_state['score']}")
print(f">> Feedback: {sim.st.session_state['feedback'][-1]}")